In [ ]:
import datacube
import time
import rasterio
import os
from matplotlib import pyplot

dc = datacube.Datacube()

In [ ]:
dc.list_products()

In [ ]:
os.environ['GDAL_NETCDF_BOTTOMUP'] = 'NO'

In [ ]:
start = time.time()

accum_prcp = dc.load(product='accum_prcp')
#accum_prcp = dc.load(product='accum_prcp',id='1311c672-c2c5-5911-a2e5-2a76f487c143')
end = time.time()
print(end - start)



#accum_prcp2 = dc.load(product='accum_prcp',id='9bec1213-ed93-5cbc-9e39-893ec90961b8')
#accum_prcp = dc.load(product='accum_prcp', output_crs='epsg:3577', resolution=(-12500,12500))

In [ ]:
accum_prcp

In [ ]:
total_day_accumprcp = accum_prcp['accum_prcp'].sum(dim='time')

In [ ]:
pyplot.imshow(total_day_accumprcp, cmap='pink')

In [ ]:
accum_prcp2 = dc.load(product='accum_prcp',id='9bec1213-ed93-5cbc-9e39-893ec90961b8')

In [ ]:
accum_prcp2

In [ ]:
pyplot.imshow(accum_prcp2['accum_prcp'].squeeze(), cmap='pink')


In [ ]:
pyplot.imshow(accum_prcp['accum_prcp'].squeeze(), cmap='pink')


In [ ]:
total_day_accumprcp = accum_prcp['accum_prcp'].sum(dim='time')

In [ ]:
start = time.time()
src = rasterio.open('NetCDF:"/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc":accum_prcp')
img=src.read(4)
end = time.time()
print(end - start)

In [ ]:
pyplot.imshow(img, cmap='pink')

In [ ]:
with rasterio.open('NetCDF:"/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc":accum_prcp', 'r') as img:
    left, bottom, right, top = [i for i in img.bounds]
    spatial_reference = str(
        str(getattr(img, 'crs_wkt', None) or img.crs.wkt))
    res = img.res
    #return left, bottom, right, top, spatial_reference, res
    print(left)
    print(bottom)
    print(right)
    print(top)
    print(spatial_reference)
    print(res)

In [ ]:
import dask
import dask.distributed

client = dask.distributed.Client(n_workers=8,
                                 threads_per_worker=1,
                                 ip='127.0.0.1')
client

In [ ]:
%%time
xx = dc.load(product='accum_prcp',
             dask_chunks={'time':6})

print("Number of chunks per band: {}x{}x{}".format(*xx.accum_prcp.data.to_delayed().shape))
display(xx)

In [ ]:
%%time
total_day_accumprcp = xx['accum_prcp'].sum(dim='time')

In [ ]:
%%time
pyplot.imshow(total_day_accumprcp, cmap='pink')